In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Importing From Google Drive

In [2]:
! conda install -y gdown

Retrieving notices: ...working... done
Channels:
 - rapidsai
 - nvidia
 - nodefaults
 - conda-forge
 - defaults
 - pytorch
Platform: linux-64
Solving environment: / - \ | / - \ | / - \ | / - \ | / - \ | done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - gdown


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2025.1.31  |       hbcca054_0         154 KB  conda-forge
    certifi-2025.1.31          |     pyhd8ed1ab_0         159 KB  conda-forge
    conda-24.11.3              |  py310hff52083_0         906 KB  conda-forge
    filelock-3.17.0            |     pyhd8ed1ab_0          17 KB  conda-forge
    gdown-5.2.0                |     pyhd8ed1ab_1          21 KB  conda-forge
    openssl-3.4.1              |       h7b32b05_0         2.8 MB  conda-forge
    ----------------------------------------

In [3]:
!gdown --id 1vJsjWEUmFJM6sOTZmVh_Zq07N9q1MY8G

/opt/conda/lib/python3.10/site-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1vJsjWEUmFJM6sOTZmVh_Zq07N9q1MY8G
From (redirected): https://drive.google.com/uc?id=1vJsjWEUmFJM6sOTZmVh_Zq07N9q1MY8G&confirm=t&uuid=b5958ec5-2066-4cf3-9d53-fa53b10b15e1
To: /kaggle/working/Summarization.zip
100%|████████████████████████████████████████| 528M/528M [00:05<00:00, 92.2MB/s]


In [4]:
import zipfile
z= zipfile.ZipFile('Summarization.zip')
z.extractall()

# Install rouge_score & evaluate

In [5]:
!pip install rouge_score

  Preparing metadata (setup.py) ... - done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=365b410abec3c668bb618206a70dfd9852bd3c5c6f1e16454ebd1e428fa15d18
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [6]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.1 MB/s eta 0:00:00


# Import Libarires

In [7]:
import pandas as pd
import numpy as np
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, get_cosine_schedule_with_warmup
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import re
from torch.optim import AdamW
import gc
import os

# Training parameters optimized for T5-base

In [8]:
# Training parameters optimized for T5-base
NUM_EPOCHS = 3
BATCH_SIZE = 2
ACCUMULATION_STEPS = 16
LEARNING_RATE = 2e-5
MAX_INPUT_LENGTH = 512
MAX_TARGET_LENGTH = 128
MODEL_NAME = 't5-base'
TRAIN_SAMPLE_SIZE = 50000
WARMUP_RATIO = 0.1
WEIGHT_DECAY = 0.01
GRADIENT_CLIP = 1.0
EPS = 1e-8

# Set up device and memory management


In [9]:
# Set up device and memory management
torch.cuda.empty_cache()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.backends.cudnn.benchmark = True

# Initialize model and tokenizer


In [10]:
# Initialize model and tokenizer
print(f"Initializing {MODEL_NAME} model and tokenizer...")
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
model = T5ForConditionalGeneration.from_pretrained(
    MODEL_NAME,
    low_cpu_mem_usage=True
)

Initializing t5-base model and tokenizer...


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

# Enable memory optimization features


In [11]:
# Enable memory optimization features
model.gradient_checkpointing_enable()
model = model.to(device)

# Load and preprocess datasets


In [12]:
# Load and preprocess datasets
print("Loading datasets...")
train_df = pd.read_csv('/kaggle/working/cnn_dailymail/train.csv')
val_df = pd.read_csv('/kaggle/working/cnn_dailymail/validation.csv')
test_df = pd.read_csv('/kaggle/working/cnn_dailymail/test.csv')

Loading datasets...


# Sample data

In [13]:
# Sample data
train_df = train_df.sample(n=min(TRAIN_SAMPLE_SIZE, len(train_df)), random_state=42)
val_df = val_df.sample(n=min(5000, len(val_df)), random_state=42)
test_df = test_df.sample(n=min(2000, len(test_df)), random_state=42)

print(f"Training samples: {len(train_df)}")
print(f"Validation samples: {len(val_df)}")
print(f"Test samples: {len(test_df)}")

Training samples: 50000
Validation samples: 5000
Test samples: 2000


# Clean text data with vectorized operations


In [14]:
# Clean text data with vectorized operations
print("Cleaning text data...")
for df in [train_df, val_df, test_df]:
    df['article'] = df['article'].astype(str).str.replace(r'\s+', ' ', regex=True)
    df['article'] = df['article'].str.replace(r'[^\w\s.,!?-]', '', regex=True)
    df['article'] = df['article'].str.strip().str[:MAX_INPUT_LENGTH * 4]
    
    df['highlights'] = df['highlights'].astype(str).str.replace(r'\s+', ' ', regex=True)
    df['highlights'] = df['highlights'].str.replace(r'[^\w\s.,!?-]', '', regex=True)
    df['highlights'] = df['highlights'].str.strip().str[:MAX_TARGET_LENGTH * 4]

Cleaning text data...


In [15]:
print("Creating features...")
all_input_ids = []
all_attention_mask = []
all_labels = []

Creating features...


# Process data in batches


In [16]:
# Process data in batches
batch_size = 1000
for i in range(0, len(train_df), batch_size):
    batch_texts = train_df['article'][i:i + batch_size].tolist()
    batch_summaries = train_df['highlights'][i:i + batch_size].tolist()
    
    inputs = tokenizer(
        ["summarize: " + text for text in batch_texts],
        max_length=MAX_INPUT_LENGTH,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )
    
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            batch_summaries,
            max_length=MAX_TARGET_LENGTH,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
    
    all_input_ids.append(inputs['input_ids'])
    all_attention_mask.append(inputs['attention_mask'])
    all_labels.append(labels['input_ids'])

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:4117: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


# Create input features in batches to save memory


In [17]:
# Create input features in batches to save memory
def create_features(texts, summaries, batch_size=1000):
    all_input_ids = []
    all_attention_mask = []
    all_labels = []
    
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i + batch_size]
        batch_summaries = summaries[i:i + batch_size]
        
        inputs = tokenizer(
            ["summarize: " + text for text in batch_texts],
            max_length=MAX_INPUT_LENGTH,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        
        with tokenizer.as_target_tokenizer():
            labels = tokenizer(
                batch_summaries,
                max_length=MAX_TARGET_LENGTH,
                padding='max_length',
                truncation=True,
                return_tensors='pt'
            )
        
        all_input_ids.append(inputs['input_ids'])
        all_attention_mask.append(inputs['attention_mask'])
        all_labels.append(labels['input_ids'])
    
    return (torch.cat(all_input_ids), 
            torch.cat(all_attention_mask), 
            torch.cat(all_labels))

# Concatenate all batches


In [18]:
# Concatenate all batches
train_input_ids = torch.cat(all_input_ids)
train_attention_mask = torch.cat(all_attention_mask)
train_labels = torch.cat(all_labels)

# Create dataset


In [19]:
# Create dataset
train_dataset = torch.utils.data.TensorDataset(
    train_input_ids,
    train_attention_mask,
    train_labels
)

# Create data loader


In [20]:
# Create data loader
train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=2,
    pin_memory=False,
    drop_last=True
)

# Initialize optimizer


In [21]:
# Initialize optimizer
optimizer = AdamW(
    model.parameters(),
    lr=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY,
    eps=EPS,
    foreach=True
)

# Setup scheduler


In [22]:
# Setup scheduler
num_training_steps = len(train_loader) * NUM_EPOCHS
scheduler = get_cosine_schedule_with_warmup(
    optimizer,
    num_warmup_steps=int(num_training_steps * WARMUP_RATIO),
    num_training_steps=num_training_steps
)

# Training loop


In [23]:
# Training loop
print("\nStarting training...")
best_loss = float('inf')
patience = 0
max_patience = 3

for epoch in range(NUM_EPOCHS):
    model.train()
    total_loss = 0
    valid_batches = 0
    
    progress_bar = tqdm(enumerate(train_loader), total=len(train_loader))
    optimizer.zero_grad(set_to_none=True)
    
    for batch_idx, (input_ids, attention_mask, labels) in progress_bar:
        # Move batch to GPU
        input_ids = input_ids.to(device, non_blocking=True)
        attention_mask = attention_mask.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True)
        
        # Replace padding token id with -100 for loss calculation
        labels[labels == tokenizer.pad_token_id] = -100
        
        # Forward pass
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )
        
        # Calculate loss
        loss = outputs.loss.mean() / ACCUMULATION_STEPS
        
        # Backward pass
        loss.backward()
        
        if (batch_idx + 1) % ACCUMULATION_STEPS == 0:
            # Clip gradients
            torch.nn.utils.clip_grad_norm_(model.parameters(), GRADIENT_CLIP)
            
            # Optimizer step
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad(set_to_none=True)
            
            valid_batches += 1
            total_loss += loss.item() * ACCUMULATION_STEPS
            avg_loss = total_loss / valid_batches
            
            progress_bar.set_postfix({
                'loss': f'{avg_loss:.4f}',
                'lr': f'{optimizer.param_groups[0]["lr"]:.2e}'
            })
        
        # Memory management
        if batch_idx % 50 == 0:
            torch.cuda.empty_cache()
            gc.collect()
    
    # Epoch-end processing
    avg_loss = total_loss / valid_batches
    print(f"\nEpoch {epoch + 1} - Average loss: {avg_loss:.4f}")
    
    # Save best model
    if avg_loss < best_loss:
        best_loss = avg_loss
        patience = 0
        torch.save(model.state_dict(), f'best_model_epoch_{epoch+1}.pt')
        print(f"New best model saved with loss: {avg_loss:.4f}")
    else:
        patience += 1
        if patience >= max_patience:
            print("\nEarly stopping triggered")
            break


Starting training...


  0%|          | 0/25000 [00:00<?, ?it/s]`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
100%|██████████| 25000/25000 [3:25:17<00:00,  2.03it/s, loss=1.9593, lr=4.17e-06]



Epoch 1 - Average loss: 1.9593
New best model saved with loss: 1.9593


100%|██████████| 25000/25000 [3:25:55<00:00,  2.02it/s, loss=1.7310, lr=8.33e-06]



Epoch 2 - Average loss: 1.7310
New best model saved with loss: 1.7310


100%|██████████| 25000/25000 [3:25:59<00:00,  2.02it/s, loss=1.6754, lr=1.25e-05]



Epoch 3 - Average loss: 1.6754
New best model saved with loss: 1.6754


# Define text cleaning function


In [24]:
# Define text cleaning function
def clean_text(text):
    text = text.strip()
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^\w\s.,!?-]', '', text)  # Keep punctuation marks for clarity
    return text

# Summary generation function


In [25]:
# Summary generation function
def generate_summary(article_text):
    model.eval()
    article_text = clean_text(article_text)

    try:
        with torch.amp.autocast('cuda'), torch.no_grad():
            inputs = tokenizer(
                "summarize: " + article_text,
                return_tensors="pt",
                max_length=MAX_INPUT_LENGTH,
                truncation=True
            ).to(device)

            outputs = model.generate(
                inputs['input_ids'],
                max_length=MAX_TARGET_LENGTH,
                min_length=50,
                num_beams=4,
                length_penalty=1.5,
                no_repeat_ngram_size=3,
                early_stopping=True,
                do_sample=True,
                temperature=0.7,
                top_p=0.92,
                top_k=50,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id
            )

        summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
        return clean_text(summary)

    except Exception as e:
        print(f"Error generating summary: {e}")
        return ""
    finally:
        torch.cuda.empty_cache()

# Test the model on multiple samples


In [26]:
# Test the model on multiple samples
print("\nTraining complete! Testing the model on 3 samples...\n")
torch.cuda.empty_cache()

# Select three sample articles
num_samples = 3
sample_articles = test_df['article'].sample(n=num_samples, random_state=42).tolist()

for idx, article in enumerate(sample_articles):
    print(f"\nSample {idx + 1}:\n")
    
    generated_summary = generate_summary(article)
    
    print("Original Article:")
    print(article[:500] + "..." if len(article) > 500 else article)
    
    print("\nGenerated Summary:")
    print(generated_summary)
    print("\n" + "=" * 80 + "\n")  # Separator line for readability


Training complete! Testing the model on 3 samples...


Sample 1:

Original Article:
A 73-year-old man is to appear in court today charged with the murder of his 95-year-old mother at her home. Police arrested David Powell after the pensioners body was discovered at her home in Doncaster Lane, Penkhull, Stoke-on-Trent on Thursday night. Although the body has not yet been formally identified, she is understood to be his mother Celilia Powell. Police arrested David Powell after the pensioners body was discovered at her home in Doncaster Lane pictured, Penkhull, Stoke-on-Trent . Ac...

Generated Summary:
Police arrested David Powell after body was discovered at her home . She is understood to be his mother Celilia Powell, 95 . Mr Powell, 73, is due to appear before magistrates at Newcastle-under-Lyme today .



Sample 2:

Original Article:
Dogs and human babies often make great companions. But a new video might have some viewers wriggling with discomfort as an infant is showered with lick

# Sample evaluation function


In [27]:
# Sample evaluation function
def evaluate_summary(original, generated):
    # Initialize scores
    scores = {
        'coherence': 0,
        'conciseness': 0,
        'relevance': 0,
        'fluency': 0,
        'coverage': 0
    }
    
    # Coherence (1-5 scale)
    # Check if the summary follows the narrative logically
    if generated in original:
        scores['coherence'] = 5
    elif len(generated) < len(original) / 2:
        scores['coherence'] = 2
    else:
        scores['coherence'] = 3
    
    # Conciseness (1-5 scale)
    # If summary is too long compared to original, lower score
    if len(generated.split()) <= len(original.split()) * 0.5:
        scores['conciseness'] = 5
    elif len(generated.split()) <= len(original.split()) * 0.75:
        scores['conciseness'] = 4
    else:
        scores['conciseness'] = 3
    
    # Relevance (1-5 scale)
    # Count how many key details from the original are present
    keywords = re.findall(r'\b\w+\b', original.lower())
    relevant_count = sum(1 for word in keywords if word in generated.lower())
    if relevant_count / len(keywords) > 0.5:
        scores['relevance'] = 5
    elif relevant_count / len(keywords) > 0.3:
        scores['relevance'] = 4
    else:
        scores['relevance'] = 2
    
    # Fluency (1-5 scale)
    # Check for grammatical correctness
    fluency_issues = sum(1 for word in generated.split() if not re.match(r'^[A-Za-z.,!?]+$', word))
    scores['fluency'] = max(0, 5 - fluency_issues)
    
    # Coverage (1-5 scale)
    # Check if all key points are covered
    if len(generated) < 20:
        scores['coverage'] = 1
    elif original in generated:
        scores['coverage'] = 5
    else:
        scores['coverage'] = 3
    
    return scores

# Testing the evaluation function


In [28]:
# Testing the evaluation function
sample_articles = [
    ("A 73-year-old man is to appear in court today charged with the murder of his 95-year-old mother at her home. Police arrested David Powell after the pensioners body was discovered at her home in Doncaster Lane, Penkhull, Stoke-on-Trent on Thursday night.", 
     "Police arrested David Powell after body was discovered at her home. She is understood to be his mother Celilia Powell, 95. Mr Powell, 73, is due to appear before magistrates at Newcastle-under-Lyme today."),

    ("Dogs and human babies often make great companions. But a new video might have some viewers wriggling with discomfort as an infant is showered with licks by his canine friend.", 
     "Video footage captured by Rumble user Sean C shows Bourbon the Doberman enthusiastically licking a baby boy on the face and nose. While some have deemed the clip cute other viewers have been less impressed by what’s going on."),

    ("One of Britains biggest babies has been born - to a mum who asked for NO pain relief. Brave Danielle Davies gave birth to Harley at nearly 12lb following an eight hour labour without any drugs.", 
     "Danielle Davies gave birth to her son Harley at nearly 12lb. She had insisted on a natural birth - not realizing she was about to deliver a massive baby. Midwives were so surprised they had to weigh him twice.")
]

# Evaluate each sample


In [29]:
# Evaluate each sample
for idx, (original, generated) in enumerate(sample_articles):
    scores = evaluate_summary(original, generated)
    print(f"Sample {idx + 1} Evaluation Scores:")
    for criterion, score in scores.items():
        print(f"{criterion.capitalize()}: {score}/5")
    print("\n" + "="*40 + "\n")

Sample 1 Evaluation Scores:
Coherence: 3/5
Conciseness: 3/5
Relevance: 5/5
Fluency: 2/5
Coverage: 3/5


Sample 2 Evaluation Scores:
Coherence: 3/5
Conciseness: 3/5
Relevance: 2/5
Fluency: 4/5
Coverage: 3/5


Sample 3 Evaluation Scores:
Coherence: 3/5
Conciseness: 3/5
Relevance: 4/5
Fluency: 3/5
Coverage: 3/5




# Final model saving


In [30]:
# Final model saving
print("\nSaving final model...")
model.save_pretrained('./summarization_model')
tokenizer.save_pretrained('./summarization_model')


Saving final model...


('./summarization_model/tokenizer_config.json',
 './summarization_model/special_tokens_map.json',
 './summarization_model/spiece.model',
 './summarization_model/added_tokens.json')

# Clean up


In [31]:
# Clean up
torch.cuda.empty_cache()
gc.collect()

171

In [32]:
print("Training completed!")

Training completed!
